In [1]:
#hide
import sys
import numpy as np
import matplotlib
from matplotlib import animation, pyplot as plt
import proplot as plot

sys.path.append('/Users/46h/Research/code/accphys/')
from tools import (
    plotting as myplt,
    animation as myanim,
    utils
)
plt.rcParams['animation.html'] = 'jshtml'
plot.rc['figure.facecolor'] = 'white'
plot.rc['savefig.dpi'] = 'figure'

# Particle-in-cell simulation
> An introduction to electromagnetic PIC solvers

- toc: true 
- badges: false
- comments: true
- categories: [physics, accelerators, simulation]
- image: images/chart-preview.png

In my research I utilize [PyORBIT](link), one of many particle-in-cell (PIC) simulation codes for particle accelerators. These codes track a beam of particles though the time-varying electromagnetic fields provided by an accelerator; we call this *external focusing*. In addition to external focusing, there are also *internal* forces to consider caused by the electromagnetic interactions between the beam particles. These are known as *space charge* forces, which refers to the charge density of the beam in free space. Space charge is significant in low-energy, high-intensity machines, so it is very important to be able to predict its effects.

The equations of motion of a single particle in the presence of external focusing and space charge are

$$\begin{bmatrix}x \\ y\end{bmatrix}'' = $$

## Introduction 

### Theoretical model 

For now, we will assume the beam is two-dimensional (2D), meaning we only need to describe its transverse evolution. We can thus think of either point charges in a beam with zero length or infinite line charges; either way 

We start from [Poisson's equation](https://en.wikipedia.org/wiki/Poisson%27s_equation) for the electric field $\mathbf{E}$ and the magnetic field $\mathbf{B}$. The first two equations determine the divergence of these fields from the charge density:

$$\nabla \cdot {\mathbf{E}} = \frac{\rho}{\varepsilon_0}$$

The 

$$\nabla \cdot {\mathbf{E}} &= \frac{\rho}{\varepsilon_0}$$
$$\nabla \cdot {\mathbf{B}} &= 0$$


$$\nabla \times {\mathbf{E}} &= \frac{\partial\mathbf{B}}{\partial{t}}$$
$$\nabla \times {\mathbf{B}} &= \mu_0\left({\mathbf{J} + \varepsilon_0\frac{\partial\mathbf{E}}{\partial{t}}}\right)$$

### Computational approach

Maxwell

## 1D solver 

## 2D solver 